In [1]:
northbound_caltrain_arrival_times = {'6:03' : int(6 * 60 + 3), '6:38' : int(6 * 60 + 38), 
                                    '6:47' : int(6 * 60 + 47), '7:24' : int(7 * 60 + 24), 
                                    '7:08' : int(7 * 60 + 8), '7:57' : int(7 * 60 + 57), 
                                    '7:51' : int(7 * 60 + 51), '8:07' : int(8 * 60 + 7), 
                                    '8:24' : int(8 * 60 + 24), '8:11' : int(8 * 60 + 11), 
                                    '8:58' : int(8 * 60 + 58), '8:53' : int(8 * 60 + 53), 
                                    '9:07' : int(9 * 60 + 7), '9:29' : int(9 * 60 + 29), 
                                    '9:11' : int(9 * 60 + 11), '9:52' : int(9 * 60 + 52), 
                                    '10:09' : int(10 * 60 + 9), '10:50' : int(10 * 60 + 50), 
                                    '11:17' : int(11 * 60 + 17), '11:48' : int(11 * 60 + 48), 
                                    '12:48' : int(12 * 60 + 48), '13:48' : int(13 * 60 + 48), 
                                    '14:48' : int(14 * 60 + 48), '15:48' : int(15 * 60 + 48), 
                                    '15:50' : int(15 * 60 + 50), '16:50' : int(16 * 60 + 50), 
                                    '17:02' : int(17 * 60 + 2), '17:36' : int(17 * 60 + 36), 
                                    '17:31' : int(17 * 60 + 31), '17:42' : int(17 * 60 + 42), 
                                    '18:06' : int(18 * 60 + 6), '17:51' : int(17 * 60 + 51), 
                                    '18:33' : int(18 * 60 + 33), '18:27' : int(18 * 60 + 27), 
                                    '18:42' : int(18 * 60 + 42), '19:06' : int(19 * 60 + 6), 
                                    '18:51' : int(18 * 60 + 51), '19:33' : int(19 * 60 + 33), 
                                    '19:27' : int(19 * 60 + 27), '19:49' : int(19 * 60 + 49), 
                                    '20:02' : int(20 * 60 + 2), '20:42' : int(20 * 60 + 42), 
                                    '21:20' : int(21 * 60 + 20), '22:20' : int(22 * 60 + 20), 
                                    '23:20' : int(23 * 60 + 20)}



southbound_caltrain_departure_times = {'4:55' : int(4 * 60) + 55, '5:25' : int(5 * 60) + 25, '6:05' : int(6 * 60) + 5, 
                                    '6:15' : int(6 * 60) + 15, '6:35' : int(6 * 60) + 35, '6:45' : int(6 * 60) + 45, 
                                    '6:59' : int(6 * 60) + 59, '7:05' : int(7 * 60) + 5, '7:15' : int(7 * 60) + 15, 
                                    '7:35' : int(7 * 60) + 35, '7:45' : int(7 * 60) + 45, '7:59' : int(7 * 60) + 59, 
                                    '8:05' : int(8 * 60) + 5, '8:15' : int(8 * 60) + 15, '8:35' : int(8 * 60) + 35, 
                                    '8:45' : int(8 * 60) + 45, '9:00' : int(9 * 60) + 0, '9:45' : int(9 * 60) + 45, 
                                    '10:00' : int(10 * 60) + 0, '11:00' : int(11 * 60) + 0, '12:00' : int(12 * 60) + 0, 
                                    '13:00' : int(13 * 60) + 0, '14:00' : int(14 * 60) + 0, '14:43' : int(14 * 60) + 43, 
                                    '15:00' : int(15 * 60) + 0, '15:34' : int(15 * 60) + 34, '16:12' : int(16 * 60) + 12, 
                                    '16:23' : int(16 * 60) + 23, '16:32' : int(16 * 60) + 32, '16:38' : int(16 * 60) + 38, 
                                    '16:58' : int(16 * 60) + 58, '17:16' : int(17 * 60) + 16, '17:27' : int(17 * 60) + 27, 
                                    '17:32' : int(17 * 60) + 32, '17:38' : int(17 * 60) + 38, '17:58' : int(17 * 60) + 58, 
                                    '18:16' : int(18 * 60) + 16, '18:23' : int(18 * 60) + 23, '18:32' : int(18 * 60) + 32, 
                                    '18:38' : int(18 * 60) + 38, '18:58' : int(18 * 60) + 58, '19:30' : int(19 * 60) + 30, 
                                    '20:30' : int(20 * 60) + 30, '21:30' : int(21 * 60) + 30, '22:40' : int(22 * 60) + 40}

# plot caltrain arrivals from SF
# for k, v in northbound_caltrain_arrival_times.items():
#     if v > am_commute_hour_begin*60 and v < am_commute_hour_finish*60:
#         ax.axvline(x=v-am_commute_hour_begin*60, linestyle='-', alpha=0.5, color='k')
# plot caltrain departures from SF
# for k, v in southbound_caltrain_departure_times.items():
#     if v > am_commute_hour_begin*60 and v < am_commute_hour_finish*60:
#         ax.axvline(x=v-am_commute_hour_begin*60, linestyle=':', alpha=0.5, color='r')




fade_alpha=0.075

# Investigation of Station Data from Bay Area Bike Share Published Data

In [2]:
%matplotlib inline

import matplotlib
import numpy as np
from scipy import stats
import scipy
import math
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import datetime

import seaborn as sns
# sns.set()
sns.set_style('whitegrid')
sns.set_context("poster")

In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

font = {'size'   : 50}
matplotlib.rc('font', **font)

TITLE_FONT_SIZE = 25
LABEL_FONT_SIZE = 15
TICK_FONT_SIZE  = 15

day_labels = ['MON','TUE','WED','THU','FRI','SAT','SUN']
day_labels_full = ['MONDAY','TUESDAY','WEDNESDAY','THURSDAY','FRIDAY','SATURDAY','SUNDAY']
month_labels = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']



sub_color = 'b'
sub_color_alt = 'm'
cust_color='r'
cust_color_alt='y'

commuter_color='g'
commuter_color_alt='#1daf1d'

commuter_am = '#ea54d9'
commuter_am_alt = '#9b8460'

commuter_pm = '#b97ccc'
commuter_pm_alt = '#f4ad3a'

figsize = (15,6)

# Load Trip Data

In [4]:
print('[%s] Loading Trip Data Data...' % datetime.datetime.now().time())

trips_df = pd.DataFrame()
trip_data_file = '../clean_data/bayareabikeshare/trip_data_extended_cleaned.csv'

# Chunk Settings
chunks = []
chunk_counter = 1
chunksize = 10000
num_chunks = math.ceil(sum(1 for row in open(trip_data_file, 'r'))/chunksize)

# import file in chunks
for chunk in pd.read_csv(trip_data_file, chunksize=chunksize, iterator=True, index_col=0, parse_dates=['start_date', 'end_date', 'forecast_time']):
    
    # append chunk to chunks list
    chunks.append(chunk)

    if chunk_counter == 1 or chunk_counter % math.ceil(num_chunks/10) == 0 or chunk_counter == num_chunks:
        print('\t\t[%s] finished chunk %s of %s' % (datetime.datetime.now().time(), chunk_counter, num_chunks))
    chunk_counter += 1

trips_df = pd.concat(chunks)
trips_df.user_type = trips_df.user_type.astype('category')
print('[%s] Complete!' % datetime.datetime.now().time())

[17:00:27.138660] Loading Trip Data Data...
		[17:00:28.972064] finished chunk 1 of 99
		[17:00:31.963748] finished chunk 10 of 99
		[17:00:34.083319] finished chunk 20 of 99
		[17:00:36.076850] finished chunk 30 of 99
		[17:00:37.872522] finished chunk 40 of 99
		[17:00:39.679946] finished chunk 50 of 99
		[17:00:41.471563] finished chunk 60 of 99
		[17:00:43.324540] finished chunk 70 of 99
		[17:00:45.182141] finished chunk 80 of 99
		[17:00:47.014054] finished chunk 90 of 99
		[17:00:48.531031] finished chunk 99 of 99
[17:00:49.559625] Complete!


In [5]:
trips_df.head(5)

,trip_id,duration,start_date,start_station_id,end_date,end_station_id,bike_id,user_type,user_zip,duration_minutes,...,humidity,precipIntensity,precipProbability,precipType,pressure,temperature,forecast_time,visibility,windBearing,windSpeed
0,4069,174,2013-08-29 09:08:00,64,2013-08-29 09:11:00,64,288,Subscriber,94114,2.900000,...,0.92,0.0,0.0,0.0,1017.06,62.63,2013-08-29 09:00:00,10.0,183.0,0.7
1,4073,1067,2013-08-29 09:24:00,66,2013-08-29 09:42:00,69,321,Subscriber,94703,17.783333,...,0.91,0.0,0.0,0.0,1017.26,62.60,2013-08-29 09:00:00,10.0,241.0,1.0
2,4074,1131,2013-08-29 09:24:00,66,2013-08-29 09:43:00,69,317,Subscriber,94115,18.850000,...,0.91,0.0,0.0,0.0,1017.26,62.60,2013-08-29 09:00:00,10.0,241.0,1.0
3,4075,1117,2013-08-29 09:24:00,66,2013-08-29 09:43:00,69,316,Subscriber,94122,18.616667,...,0.91,0.0,0.0,0.0,1017.26,62.60,2013-08-29 09:00:00,10.0,241.0,1.0
4,4076,1118,2013-08-29 09:25:00,66,2013-08-29 09:43:00,69,322,Subscriber,94597,18.633333,...,0.91,0.0,0.0,0.0,1017.26,62.60,2013-08-29 09:00:00,10.0,241.0,1.0


In [145]:
subscriber_trips = trips_df[trips_df.user_type == 'Subscriber'].copy()
customer_trips   = trips_df[trips_df.user_type == 'Customer'].copy()

commuter_trips = subscriber_trips[subscriber_trips.start_date.dt.dayofweek < 5].copy()
commuter_trips = commuter_trips[((commuter_trips.start_date.dt.hour >= 7) & (commuter_trips.start_date.dt.hour < 10)) | 
                                ((commuter_trips.start_date.dt.hour >= 16) & (commuter_trips.start_date.dt.hour < 19))].copy()

In [146]:
def monthly_resample_sum(df=None):
    resampled_count = df[-((df.start_date.dt.year == 2013) & (df.start_date.dt.month == 8))]
    resampled_count = resampled_count.groupby([resampled_count.start_date.dt.date]).count()['trip_id'].to_frame()
    resampled_count.reset_index(inplace=True)
    resampled_count.start_date = pd.to_datetime(resampled_count.start_date)
    resampled_count.set_index(['start_date'], inplace=True)
    resampled_count = resampled_count.resample('1M').sum()
    
    return resampled_count.trip_id

In [147]:
trip_counts = pd.DataFrame()
trip_counts['customer_trips'] = monthly_resample_sum(df=customer_trips)
trip_counts['subscriber_trips'] = monthly_resample_sum(df=subscriber_trips)
trip_counts['commuter_trips'] = monthly_resample_sum(df=commuter_trips)
trip_counts['non_commuter_trips'] = trip_counts.subscriber_trips - trip_counts.commuter_trips

In [148]:
trip_counts

,customer_trips,subscriber_trips,commuter_trips,non_commuter_trips
start_date,,,,
2013-09-30,9464,15766,7517,8249
2013-10-31,6112,22977,12502,10475
2013-11-30,4471,19729,10483,9246
2013-12-31,3229,16656,8870,7786
2014-01-31,3402,21015,11936,9079
2014-02-28,2449,16565,9629,6936
2014-03-31,3994,20583,11695,8888
2014-04-30,4075,22134,13210,8924
2014-05-31,5118,23375,13967,9408


In [149]:
x_ticks = []
for idx in trip_counts.index:
    x_ticks.append('%s %d' % (month_labels[idx.month-1], idx.year))
trip_counts.reset_index(inplace=True, drop=True)

In [150]:
len(x_ticks)

36

In [167]:
ax = trip_counts.subscriber_trips.plot(color=sub_color, linestyle='-.', marker='o', label='Subscribers', figsize=(18,6))
trip_counts.customer_trips.plot(ax=ax, color=cust_color, linestyle='-.', marker='o', label='Customers')
trip_counts.commuter_trips.plot(ax=ax, color=commuter_color, linestyle='-.', marker='o', label='Commuters')
trip_counts.non_commuter_trips.plot(ax=ax, color=commuter_color_alt, linestyle='-.', marker='o', label='Non Commuters')

ax.set_title('Trips By Month', size=TITLE_FONT_SIZE, weight='bold')
ax.set_ylabel('Number of Trips', size=LABEL_FONT_SIZE, weight='bold')

y_min = 0
y_max = 35000
y_interval = 2500
ax.set_ylim([y_min, y_max])
y_ticks = [x*y_interval for x in range(y_min, int(y_max/y_interval)+1)]
ax.set_yticks(y_ticks)


ax.set_xticks([x for x in range(0, len(x_ticks))])
ax.set_xticklabels(x_ticks, rotation=90, size=10)
ax.set_xlabel('')

year_markers = [3, 15, 27]
for x in year_markers:
    ax.axvline(x=x, color='k', alpha=0.35)

ax.legend(['Subscribers', 'Customers', 'Commuters', 'Non Commuters'], loc=2, frameon=True)
# plt.show()
plt.savefig('../charts/user_trends/month_to_month.png')
plt.close()

In [ ]:
subscriber_trips = trips_df[trips_df.user_type == 'Subscriber'].copy()
subscriber_trips.reset_index(inplace=True, drop=True)
subscriber_trips.shape

In [ ]:
sub_30min  = subscriber_trips[subscriber_trips.duration_minutes <= 30.0].copy()
sub_30min.reset_index(inplace=True, drop=True)
print(sub_30min.shape)
over_30min = subscriber_trips[subscriber_trips.duration_minutes > 30.0].copy()
over_30min.reset_index(inplace=True, drop=True)
print(over_30min.shape)

In [ ]:
over_30min.shape[0] / subscriber_trips.shape[0]*100. 

In [ ]:
sub_30min.shape[0] / subscriber_trips.shape[0]*100. 

## Utility Functions

In [ ]:
def zip_to_landmark(zip_code):
    ''' Return zipcode for given landmark'''
    if zip_code == 94107:
        return 'San Francisco'
    if zip_code == 94063:
        return 'Redwood City'
    if zip_code == 94301:
        return 'Palo Alto'
    if zip_code == 94041:
        return 'Mountain View'
    if zip_code == 95113:
        return 'San Jose'
    return False

In [ ]:

ordered_zipcodes = [94107, 95113, 94041, 94301, 94063]

commuter_am = '#ea54d9'
commuter_am_alt = '#9b8460'

commuter_pm = '#b97ccc'
commuter_pm_alt = '#f4ad3a'


figsize = (15,6)

# Cleanup
<p>Prune to only Commuter Trips, in all regions</p>

<ul>
    <li>user_type : subscriber</li>
    <li>start and end times are between 7am and 10am, or between 4pm and 7pm</li>
</ul>

In [ ]:
subscriber_trips_df = trips_df[trips_df.user_type == 'Subscriber']

morning_commuter_trips_df = subscriber_trips_df[((subscriber_trips_df.start_date.dt.hour >= 7) & (subscriber_trips_df.start_date.dt.hour < 10) | 
                                                (subscriber_trips_df.end_date.dt.hour >= 7) & (subscriber_trips_df.end_date.dt.hour < 10) )&
                                                (subscriber_trips_df.start_date.dt.dayofweek < 5)]
evening_commuter_trips_df = subscriber_trips_df[((subscriber_trips_df.start_date.dt.hour >= 16) & (subscriber_trips_df.start_date.dt.hour < 19) | 
                                                (subscriber_trips_df.end_date.dt.hour >= 16) & (subscriber_trips_df.end_date.dt.hour < 19) )&
                                                (subscriber_trips_df.start_date.dt.dayofweek < 5)]

commuter_trips_df = pd.concat([morning_commuter_trips_df, evening_commuter_trips_df])
commuter_trips_df.reset_index(inplace=True, drop=True)
commuter_trips_df.head(3)

In [ ]:
commuter_trips_df.info()

## Analysis 1 - Explore Station Usage
<p>Find Most Start and End Terminals</p>
<p></p>
<p></p>

In [ ]:
popular_stations_df = pd.DataFrame()

# count trips starting at each terminal 
popular_stations_df = commuter_trips_df.groupby('start_terminal')['trip_id'].count().to_frame()
popular_stations_df.columns = ['start_count']

# calculate share of all trip starts for each terminal
popular_stations_df['start_trip_share'] = popular_stations_df['start_count'] / popular_stations_df['start_count'].sum() * 100.

# count trips ending at each terminal
popular_stations_df['end_count'] = commuter_trips_df.groupby('end_terminal')['trip_id'].count().to_frame()

# calculate share of all trip ends for each terminal
popular_stations_df['end_trip_share'] = popular_stations_df['end_count'] / popular_stations_df['end_count'].sum() * 100.

# set index to terminal id
popular_stations_df.index.name = 'terminal_id'
popular_stations_df.head()

# Graphical EDA of Start and End Terminal Trip Counts by Region

In [ ]:
top_start_stations_in_region = {}
top_end_stations_in_region = {}

for zip_code in ordered_zipcodes:
    
    df = pd.DataFrame()
    
    tmp = commuter_trips_df[(commuter_trips_df.start_zip == zip_code) & (commuter_trips_df.end_zip == zip_code)]

    df = tmp.groupby('start_terminal')['trip_id'].count().to_frame()
    df.columns = ['start_count']

    df['start_share'] = popular_stations_df['start_count'] / df['start_count'].sum() * 100.
    df['end_count'] = tmp.groupby('end_terminal')['trip_id'].count().to_frame()
    
    df['end_share'] = popular_stations_df['end_count'] / df['end_count'].sum() * 100.
    
    df.index.name = 'terminal_id'
    
    # append top n to top_stations_in_region
    top_start_stations_in_region[zip_to_landmark(zip_code)] = list(df.sort_values(['start_share'], ascending=False)['start_share'].index)
    top_end_stations_in_region[zip_to_landmark(zip_code)] = list(df.sort_values(['end_share'], ascending=False)['end_share'].index)
    
    plt.subplots(figsize=(15,6))

    ax = sns.barplot(x=df.index, y = 'start_count', data=df, color='b', alpha=0.5, label='trips starting')
    sns.barplot(x=df.index, y = 'end_count', data=df, color='r', alpha=0.5, label='trips ending', ax=ax)

    title = '%s Terminal Usage' % zip_to_landmark(zip_code)
    ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')
    ax.set_xlabel('Terminal ID', size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel('Number of Trips', size=LABEL_FONT_SIZE, weight='bold')
    
    # set y axis
    y_interval = df.max().max()/5000
    
    if y_interval > 10:
        y_interval = 10000
    elif y_interval <= 10 and y_interval > 1:
        y_interval = 1000
    else:
        y_interval = 500

    y_max = -np.inf
    y_max = df.max().max()
    y_max = math.ceil(y_max/y_interval) * y_interval
    
    ax.set_ylim([0, y_max])
    ax.set_yticks([y for y in range(0, y_max+y_interval, y_interval)])
    ax.set_ylabel('Number of Trips', size=LABEL_FONT_SIZE, weight='bold')
    
    ax.legend(loc=1, frameon=True)
        
    image_title = '../charts/investigation_stations/Investigation_%s.png' % title.replace(' ', '_').replace('_-_', '_')
    plt.savefig(image_title)

    plt.show()

## Graphical EDA of Start and End Terminal Trip Counts by Region - Day Count Adjusted

## Load Station Data

In [ ]:
print('[%s] Loading Status Data...' % datetime.datetime.now().time())

station_df = pd.DataFrame()
station_file = '../clean_data/bayareabikeshare/station_data_cleaned.csv'

# Chunk Settings
chunks = []
chunk_counter = 1
chunksize = 10000
num_chunks = math.ceil(sum(1 for row in open(station_file, 'r'))/chunksize)

# import file in chunks
for chunk in pd.read_csv(station_file, chunksize=chunksize, iterator=True, index_col=0, parse_dates=['first_service_date', 'last_service_date']):
    
    # append chunk to chunks list
    chunks.append(chunk)

    if chunk_counter == 1 or chunk_counter % math.ceil(num_chunks/10) == 0 or chunk_counter == num_chunks:
        print('\t\t[%s] finished chunk %s of %s' % (datetime.datetime.now().time(), chunk_counter, num_chunks))
    chunk_counter += 1

station_df = pd.concat(chunks)

print('[%s] Complete!' % datetime.datetime.now().time())

In [ ]:
station_df.info()

In [ ]:
station_df.head()

# Station Data with appeneded start and end trip counts

In [ ]:
def weekdays_in_service(row):
    return pd.date_range(start=row.first_service_date, end=row.last_service_date, freq='B').shape[0]

In [ ]:
station_df['commuter_days'] = station_df.apply(lambda row: weekdays_in_service (row), axis=1)

In [ ]:
station_df.head()

In [ ]:
def trips_per_day(row, trips_df=None, which=''):
    '''Get numebr of trips started or ended at each terminal between terminals first and last day of service only'''
    terminal_id = row.station_id
    terminal_first_service_date = row.first_service_date
    terminal_last_service_date  = row.last_service_date
    
    if which == 'start':
        tmp = trips_df[(trips_df.start_terminal == terminal_id) & (trips_df.start_date >= terminal_first_service_date) 
                       & (trips_df.start_date < terminal_last_service_date)]
        return tmp.shape[0]
    if which == 'end':
        tmp = trips_df[(trips_df.end_terminal == terminal_id) & (trips_df.start_date >= terminal_first_service_date) 
                       & (trips_df.start_date < terminal_last_service_date)]
        return tmp.shape[0]

In [ ]:
# for each station, get count of rides started from that station between first and last service dates

# append data from only trips by commuters
station_df['commuter_start_trip_count'] = station_df.apply(lambda row: trips_per_day (row, trips_df=commuter_trips_df, which='start'), axis=1)
station_df['commuter_end_trip_count']   = station_df.apply(lambda row: trips_per_day (row, trips_df=commuter_trips_df, which='end'), axis=1)

# append data from all trips
station_df['start_trip_count'] = station_df.apply(lambda row: trips_per_day (row, trips_df=trips_df, which='start'), axis=1)
station_df['end_trip_count']   = station_df.apply(lambda row: trips_per_day (row, trips_df=trips_df, which='end'), axis=1)

In [ ]:
# Calculate Trips per day
station_df['commuter_trip_starts_per_day'] = station_df['commuter_start_trip_count'] / station_df['commuter_days']
station_df['commuter_trip_ends_per_day']   = station_df['commuter_end_trip_count'] / station_df['commuter_days']

station_df['trip_starts_per_day'] = station_df['start_trip_count'] / station_df['days_in_service']
station_df['trip_ends_per_day']   = station_df['end_trip_count'] / station_df['days_in_service']

In [ ]:
station_df.tail()

In [ ]:
for zip_code in ordered_zipcodes:
    
    df = station_df[(station_df.zip_code == zip_code)]
    
    

    plt.subplots(figsize=(15,6))

    ax = sns.barplot(x=df.index, y = 'commuter_trip_starts_per_day', data=df, color='b', alpha=0.5, label='trips starting')
    sns.barplot(x=df.index, y = 'commuter_trip_ends_per_day', data=df, color='r', alpha=0.5, label='trips ending', ax=ax)

    
    ax.set_xticklabels(df.station_id)
    
    title = '%s Terminal Usage Per Day' % zip_to_landmark(zip_code)
    ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')
    ax.set_xlabel('Terminal ID', size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel('Number of Trips', size=LABEL_FONT_SIZE, weight='bold')
    
    
    y_max = -np.inf
    y_max = df['commuter_trip_ends_per_day'].max()
    
    if df['commuter_trip_starts_per_day'].max() > y_max:
        df['commuter_trip_ends_per_day'].max()
    
    y_interval = y_max/10
    
    if y_interval > 4:
        y_interval = 10
    elif y_interval < 4 and y_interval > 3:
        y_interval = 5
    else:
        y_interval = 1

    y_max = math.ceil(y_max/y_interval) * y_interval
    
    if y_max == 2:
        y_max = 3
        
    ax.set_ylim([0, y_max])
    
    ax.set_yticks([y for y in range(0, y_max+y_interval, y_interval)])
    ax.set_ylabel('Share of Trips', size=LABEL_FONT_SIZE, weight='bold')
    ax.legend(loc=1, frameon=True)
        
    image_title = '../charts/investigation_stations/Investigation_%s.png' % title.replace(' ', '_').replace('_-_', '_')
    plt.savefig(image_title)

    plt.show()

# Identify Popular  Commuter Routes
<p> A Route is a trip from Terminal A to Terminal B</p>

In [ ]:
station_df.head()

In [ ]:
for zip_code in ordered_zipcodes:
    
    zip_stations = sorted(pd.unique(station_df[station_df.zip_code == zip_code]['station_id']))
    
    index   = [x for x in zip_stations]
    columns = [x for x in zip_stations]

    trip_grid = pd.DataFrame(index=index, columns=columns)
    
    # create grid of trips from start to end terminal counts
    for sid in zip_stations:
        for eid in zip_stations:
            # count trips between terminals
            route_trip_count = commuter_trips_df[(commuter_trips_df.start_terminal == sid) & (commuter_trips_df.end_terminal == eid)].shape[0]
            trip_grid.loc[eid, sid] = route_trip_count

    trip_grid = trip_grid.iloc[::-1]
    
    # drop nan rows and columns
    trip_grid.dropna(axis=0, how='all', inplace=True)
    trip_grid.dropna(axis=1, how='all', inplace=True)
    
    mask = trip_grid == 0
    grid_dims = 15
    fig, ax = plt.subplots(figsize=(grid_dims*1.5, grid_dims))
    
    sns.heatmap(data=trip_grid, linewidths=.5, square=True, cmap=sns.color_palette("OrRd", 20), mask=mask, ax=ax, cbar_kws={"shrink": .75}, cbar=True)
    
    ax.set_xlabel('Start Terminal', size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel('End Terminal', size=LABEL_FONT_SIZE, weight='bold')
    
    title = '%s Commuter Route Heatmap' % zip_to_landmark(zip_code)
    ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')

    image_title = '../charts/investigation_stations/Investigation_%s.png' % title.replace(' ', '_').replace('_-_', '_')
    plt.savefig(image_title)

    plt.show()

In [ ]:
for zip_code in ordered_zipcodes:
    
    zip_stations = sorted(pd.unique(station_df[station_df.zip_code == zip_code]['station_id']))
    
    index   = [x for x in zip_stations]
    columns = [x for x in zip_stations]

    trip_grid = pd.DataFrame(index=index, columns=columns)
    
    # create grid of trips from start to end terminal counts
    for sid in zip_stations:
        for eid in zip_stations:
            # count trips between terminals
            route_trip_count = commuter_trips_df[(commuter_trips_df.start_terminal == sid) & (commuter_trips_df.end_terminal == eid)].shape[0]
            trip_grid.loc[eid, sid] = route_trip_count

    trip_grid = trip_grid.iloc[::-1]
    
    # drop nan rows and columns
    trip_grid.dropna(axis=0, how='all', inplace=True)
    trip_grid.dropna(axis=1, how='all', inplace=True)
    
    # normalize trip grid    
    trip_grid = (trip_grid - trip_grid.mean().mean()) / (trip_grid.max().max() - trip_grid.min().min())

    mask = trip_grid == 0
    grid_dims = 15
    fig, ax = plt.subplots(figsize=(grid_dims*1.5, grid_dims))
    
    sns.heatmap(data=trip_grid, linewidths=.5, square=True, cmap=sns.color_palette("OrRd", 20), mask=mask, ax=ax, cbar_kws={"shrink": .75}, cbar=True)
    
    ax.set_xlabel('Start Terminal', size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel('End Terminal', size=LABEL_FONT_SIZE, weight='bold')
    
    title = '%s Commuter Route Heatmap Normalized' % zip_to_landmark(zip_code)
    ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')

    image_title = '../charts/investigation_stations/Investigation_%s.png' % title.replace(' ', '_').replace('_-_', '_')
    plt.savefig(image_title)

    plt.show()

# Identify Top Start and End Stations Used by Commuters

In [ ]:
station_df.head(3)

In [ ]:
for zip_code in ordered_zipcodes:
    print(zip_to_landmark(zip_code))
    print('-' * 80)
    
    df = pd.DataFrame()
    df = station_df[station_df.zip_code == zip_code].copy()
    df.sort_values('commuter_start_trip_count', ascending=False, inplace=True)
    df_trip_count = df.commuter_start_trip_count.head(10).tolist()
    df_station_name = df.name.head(10).tolist()
    
    df = df[['station_id', 'name', 'commuter_start_trip_count']].head(10)
    top_stations = df.station_id.tolist()
    
    print('Top Commuter Start Terminals by Total Trip Count')
    print('\tID -  Trips - Trip Share')
    for i, sid in enumerate(top_stations):
        trip_share = df_trip_count[i]/sum(df_trip_count)*100.
        trip_share = '{:2.4f}'.format(trip_share)
        
        print('\t%s - %s - %s - %s' % (str(sid).rjust(2), str(df_trip_count[i]).rjust(6), trip_share.rjust(7),  df_station_name[i]))
    
    print('')
    df = pd.DataFrame()
    df = station_df[station_df.zip_code == zip_code].copy()
    df.sort_values('commuter_end_trip_count', ascending=False, inplace=True)
    df_trip_count = df.commuter_end_trip_count.head(10).tolist()
    df_station_name = df.name.head(10).tolist()
    
    df = df[['station_id', 'name', 'commuter_end_trip_count']].head(10)
    top_stations = df.station_id.tolist()
    
    print('Top Commuter End Terminals by Total Trip Count')
    print('\tID -  Trips - Trip Share')
    for i, sid in enumerate(top_stations):
        trip_share = df_trip_count[i]/sum(df_trip_count)*100.
        trip_share = '{:2.4f}'.format(trip_share)
        
        print('\t%s - %s - %s - %s' % (str(sid).rjust(2), str(df_trip_count[i]).rjust(6), trip_share.rjust(7),  df_station_name[i]))

    print('')

# Split Trips into Morning and Evening Commute Hours

In [ ]:
morning_commuter_trips_df.head(3)

In [ ]:
evening_commuter_trips_df.head(3)

## Append Morning Commute Station Data

In [ ]:
# for each station, get count of rides started from that station between first and last service dates

# append data from only trips by commuters
station_df['morning_commuter_start_trip_count'] = station_df.apply(lambda row: trips_per_day (row, trips_df=morning_commuter_trips_df, which='start'), axis=1)
station_df['morning_commuter_end_trip_count']   = station_df.apply(lambda row: trips_per_day (row, trips_df=morning_commuter_trips_df, which='end'), axis=1)

# Calculate Trips per day
station_df['morning_commuter_trip_starts_per_day'] = station_df['morning_commuter_start_trip_count'] / station_df['commuter_days']
station_df['morning_commuter_trip_ends_per_day']   = station_df['morning_commuter_end_trip_count'] / station_df['commuter_days']

## Append Evening Commute Station Data

In [ ]:
# for each station, get count of rides started from that station between first and last service dates

# append data from only trips by commuters
station_df['evening_commuter_start_trip_count'] = station_df.apply(lambda row: trips_per_day (row, trips_df=evening_commuter_trips_df, which='start'), axis=1)
station_df['evening_commuter_end_trip_count']   = station_df.apply(lambda row: trips_per_day (row, trips_df=evening_commuter_trips_df, which='end'), axis=1)

# Calculate Trips per day
station_df['evening_commuter_trip_starts_per_day'] = station_df['evening_commuter_start_trip_count'] / station_df['commuter_days']
station_df['evening_commuter_trip_ends_per_day']   = station_df['evening_commuter_end_trip_count'] / station_df['commuter_days']

In [ ]:
station_df.columns

## Plot Heatmap of Morning Routes

In [ ]:
morning_commuter_trips_df.head(3)

In [ ]:
for zip_code in ordered_zipcodes:

    df = morning_commuter_trips_df[(morning_commuter_trips_df.start_zip == zip_code) & (morning_commuter_trips_df.end_zip == zip_code)].copy()

    start_stations = sorted(pd.unique(df.start_terminal))
    end_stations = sorted(pd.unique(df.end_terminal))

    stations = start_stations

    for s in end_stations:
        if s not in stations:
            stations.append(s)

    index   = [x for x in stations]
    columns = [x for x in stations]

    trip_grid = pd.DataFrame(index=index, columns=columns)

    # create grid of trips from start to end terminal counts
    for sid in stations:
        for eid in stations:
            # count trips between terminals
            route_trip_count = df[(df.start_terminal == sid) & (df.end_terminal == eid)].shape[0]
            trip_grid.loc[eid, sid] = route_trip_count

    trip_grid = trip_grid.iloc[::-1]

    # drop nan rows and columns
    trip_grid.dropna(axis=0, how='all', inplace=True)
    trip_grid.dropna(axis=1, how='all', inplace=True)

    mask = trip_grid == 0
    grid_dims = 15
    fig, ax = plt.subplots(figsize=(grid_dims*1.5, grid_dims))

    sns.heatmap(data=trip_grid, linewidths=.5, square=True, cmap=sns.color_palette("OrRd", 20), mask=mask, ax=ax, cbar_kws={"shrink": .75}, cbar=True)

    ax.set_xlabel('Start Terminal', size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel('End Terminal', size=LABEL_FONT_SIZE, weight='bold')

    title = '%s Morning Commuter Route Heatmap' % zip_to_landmark(zip_code)
    ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')

    image_title = '../charts/investigation_stations/Investigation_%s.png' % title.replace(' ', '_').replace('_-_', '_')
    plt.savefig(image_title)

    plt.show()

In [ ]:
for zip_code in ordered_zipcodes:
    print(zip_to_landmark(zip_code))
    print('-' * 80)
    
    df = pd.DataFrame()
    df = station_df[station_df.zip_code == zip_code].copy()
    df.sort_values('morning_commuter_start_trip_count', ascending=False, inplace=True)
    df_trip_count = df.morning_commuter_start_trip_count.head(10).tolist()
    df_station_name = df.name.head(10).tolist()
    
    df = df[['station_id', 'name', 'morning_commuter_start_trip_count']].head(10)
    top_stations = df.station_id.tolist()
    
    print('Top Morning Commuter Start Terminals by Total Trip Count')
    print('\tID -  Trips - Trip Share')
    for i, sid in enumerate(top_stations):
        trip_share = df_trip_count[i]/sum(df_trip_count)*100.
        trip_share = '{:2.4f}'.format(trip_share)
        
        print('\t%s - %s - %s - %s' % (str(sid).rjust(2), str(df_trip_count[i]).rjust(6), trip_share.rjust(7),  df_station_name[i]))
    
    print('')
    df = pd.DataFrame()
    df = station_df[station_df.zip_code == zip_code].copy()
    df.sort_values('morning_commuter_end_trip_count', ascending=False, inplace=True)
    df_trip_count = df.morning_commuter_end_trip_count.head(10).tolist()
    df_station_name = df.name.head(10).tolist()
    
    df = df[['station_id', 'name', 'morning_commuter_end_trip_count']].head(10)
    top_stations = df.station_id.tolist()
    
    print('Top Morning Commuter End Terminals by Total Trip Count')
    print('\tID -  Trips - Trip Share')
    for i, sid in enumerate(top_stations):
        trip_share = df_trip_count[i]/sum(df_trip_count)*100.
        trip_share = '{:2.4f}'.format(trip_share)
        
        print('\t%s - %s - %s - %s' % (str(sid).rjust(2), str(df_trip_count[i]).rjust(6), trip_share.rjust(7),  df_station_name[i]))

    print('')

## Plot Heatmap of Evening Routes

In [ ]:
evening_commuter_trips_df.head(3)

In [ ]:
for zip_code in ordered_zipcodes:

    df = evening_commuter_trips_df[(evening_commuter_trips_df.start_zip == zip_code) & (evening_commuter_trips_df.end_zip == zip_code)].copy()

    start_stations = sorted(pd.unique(df.start_terminal))
    end_stations = sorted(pd.unique(df.end_terminal))

    stations = start_stations

    for s in end_stations:
        if s not in stations:
            stations.append(s)

    index   = [x for x in stations]
    columns = [x for x in stations]

    trip_grid = pd.DataFrame(index=index, columns=columns)

    # create grid of trips from start to end terminal counts
    for sid in stations:
        for eid in stations:
            # count trips between terminals
            route_trip_count = df[(df.start_terminal == sid) & (df.end_terminal == eid)].shape[0]
            trip_grid.loc[eid, sid] = route_trip_count

    trip_grid = trip_grid.iloc[::-1]

    # drop nan rows and columns
    trip_grid.dropna(axis=0, how='all', inplace=True)
    trip_grid.dropna(axis=1, how='all', inplace=True)

    mask = trip_grid == 0
    grid_dims = 15
    fig, ax = plt.subplots(figsize=(grid_dims*1.5, grid_dims))

    sns.heatmap(data=trip_grid, linewidths=.5, square=True, cmap=sns.color_palette("OrRd", 20), mask=mask, ax=ax, cbar_kws={"shrink": .75}, cbar=True)

    ax.set_xlabel('Start Terminal', size=LABEL_FONT_SIZE, weight='bold')
    ax.set_ylabel('End Terminal', size=LABEL_FONT_SIZE, weight='bold')

    title = '%s Evening Commuter Route Heatmap' % zip_to_landmark(zip_code)
    ax.set_title(title, size=TITLE_FONT_SIZE, weight='bold')

    image_title = '../charts/investigation_stations/Investigation_%s.png' % title.replace(' ', '_').replace('_-_', '_')
    plt.savefig(image_title)

    plt.show()

In [ ]:
for zip_code in ordered_zipcodes:
    print(zip_to_landmark(zip_code))
    print('-' * 80)
    
    df = pd.DataFrame()
    df = station_df[station_df.zip_code == zip_code].copy()
    df.sort_values('evening_commuter_start_trip_count', ascending=False, inplace=True)
    df_trip_count = df.evening_commuter_start_trip_count.head(10).tolist()
    df_station_name = df.name.head(10).tolist()
    
    df = df[['station_id', 'name', 'evening_commuter_start_trip_count']].head(10)
    top_stations = df.station_id.tolist()
    
    print('Top Evening Commuter Start Terminals by Total Trip Count')
    print('\tID -  Trips - Trip Share')
    for i, sid in enumerate(top_stations):
        trip_share = df_trip_count[i]/sum(df_trip_count)*100.
        trip_share = '{:2.4f}'.format(trip_share)
        
        print('\t%s - %s - %s - %s' % (str(sid).rjust(2), str(df_trip_count[i]).rjust(6), trip_share.rjust(7),  df_station_name[i]))
    
    print('')
    df = pd.DataFrame()
    df = station_df[station_df.zip_code == zip_code].copy()
    df.sort_values('evening_commuter_end_trip_count', ascending=False, inplace=True)
    df_trip_count = df.evening_commuter_end_trip_count.head(10).tolist()
    df_station_name = df.name.head(10).tolist()
    
    df = df[['station_id', 'name', 'evening_commuter_end_trip_count']].head(10)
    top_stations = df.station_id.tolist()
    
    print('Top Evening Commuter End Terminals by Total Trip Count')
    print('\tID -  Trips - Trip Share')
    for i, sid in enumerate(top_stations):
        trip_share = df_trip_count[i]/sum(df_trip_count)*100.
        trip_share = '{:2.4f}'.format(trip_share)
        
        print('\t%s - %s - %s - %s' % (str(sid).rjust(2), str(df_trip_count[i]).rjust(6), trip_share.rjust(7),  df_station_name[i]))

    print('')

# Station Balance at Top Commuter Stations
<ul>Top Commuter Stations
	<li><b>Station 70 :</b> San Francisco Caltrain (Townsend at 4th)</li>
	<li><b>Station 69 :</b> San Francisco Caltrain 2 (330 Townsend)</li>
	<li><b>Station 50 :</b> Harry Bridges Plaza (Ferry Building)</li>
	<li><b>Station 55 :</b> Temporary Transbay Terminal (Howard at Beale)</li>
	<li><b>Station 74 :</b> Steuart at Market</li>
</ul>